In [2]:
import tensorflow as tf
from tensorflow.keras import Model,activations
from tensorflow.keras.activations import relu
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,BatchNormalization,Input,ZeroPadding2D,BatchNormalization,Add,AveragePooling2D,Flatten,Dense,Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory # # Method 2 to load Dataset in model

2021-07-19 17:51:55.649329: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-19 17:51:55.649351: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=7,
    zoom_range=(0.95,0.95),
    horizontal_flip=False,
    vertical_flip=False,
    data_format='channels_last',
    validation_split=0.0,
    dtype=tf.float32
)

In [4]:
training_data = datagen.flow_from_directory(
    'birds/train',
    target_size=(224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

Found 4936 images belonging to 34 classes.


In [5]:
def identity_block(x,Filters,block_number):
    # saving previous model which will add afterwards. 
    x_shortcut = x
    
    # =========================================First Block====================================================
    # First Convolution Block
    name1="Conv "+ str(block_number)+"a" 
    x = Conv2D(Filters[0],1)(x)
    # Batch normalisation
    name="BatchNormalisation "+ str(block_number)+"_a" 
    x = BatchNormalization(axis=3)(x)
    # Activation Function
    x = Activation('relu')(x)
    #=========================================Second Block===================================================
    # Second Convolution Block
    name="Conv "+ str(block_number)+"_b" 
    x = Conv2D(Filters[1],3,padding='same')(x)
    # Batch normalisation
    name="BatchNormalisation "+ str(block_number)+"_b" 
    x = BatchNormalization(axis=3)(x)
    # Activation Function
    x = Activation('relu')(x)
    #=========================================Third Block===================================================
    # Third Convolution Block
    name="Conv "+ str(block_number)+"_c" 
    x = Conv2D(Filters[2],1)(x)
    # Batch normalisation
    name="BatchNormalisation "+ str(block_number)+"_c" 
    x = BatchNormalization(axis=3)(x)
    
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation
    x = Add()([x,x_shortcut])
    x = Activation('relu')(x)
    return x
    

In [6]:
def convolutional_block(x,Filters,block_number):
    if block_number>1:
        s=2
    if block_number==1:
        s=1
    # saving previous model which will add afterwards. 
    x_shortcut = x
    
    # =========================================First Block====================================================
    # First Convolution Block
    name1="Conv "+ str(block_number)+"a" 
    x = Conv2D(Filters[0],1,strides=(s,s))(x)
    # Batch normalisation
    name="BatchNormalisation "+ str(block_number)+"_a" 
    x = BatchNormalization(axis=3)(x)
    # Activation Function
    x = Activation('relu')(x)
    #=========================================Second Block===================================================
    # Second Convolution Block
    name="Conv "+ str(block_number)+"_b" 
    x = Conv2D(Filters[1],3,padding='same')(x)
    # Batch normalisation
    name="BatchNormalisation "+ str(block_number)+"_b" 
    x = BatchNormalization(axis=3)(x)
    # Activation Function
    x = Activation('relu')(x)
    #=========================================Third Block===================================================
    # Third Convolution Block
    name="Conv "+ str(block_number)+"_c" 
    x = Conv2D(Filters[2],1)(x)
    # Batch normalisation
    name="BatchNormalisation "+ str(block_number)+"_c" 
    x = BatchNormalization(axis=3)(x)
    
    # Now x_shortcut has dimensions(55,55,64) and next has (55, 55, 256) so we will to modify something that is
    # we have to create Conv2D and pass 256 filters as it will equal up he thing. As i am future we have to add up
    # previous block stage and Third Block of previous stage.
    x_shortcut = Conv2D(Filters[2],1)(x_shortcut)
    x_shortcut = BatchNormalization(axis=3)(x)
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation
    x = Add()([x,x_shortcut])
    x = Activation('relu')(x)
    return x

In [10]:
def resnet152(input_size):

    X_input = Input(shape=input_size)
    print(f'Shape of X_input {X_input.shape}')
    
    # ZeroPadding 
    X = ZeroPadding2D((3,3))(X_input)
    
    # conv1
    X=Conv2D(64,7,strides=(2,2),name="Conv1")(X)
    
    # MaxPooling2D
    X = MaxPool2D((3,3),strides=(2,2))(X)
    
    #Conv2_x
    X = convolutional_block(X,Filters=[64,64,256],block_number=1)
    X = identity_block(X,Filters=[64,64,256],block_number=1)
    X = identity_block(X,Filters=[64,64,256],block_number=1)
    
    #Conv3_x
    X = convolutional_block(X,Filters=[128,128,512],block_number=2)
    X = identity_block(X,Filters=[128,128,512],block_number=2)
    X = identity_block(X,Filters=[128,128,512],block_number=2)
    X = identity_block(X,Filters=[128,128,512],block_number=2)
    
    #Conv4_x
    X = convolutional_block(X,Filters=[256,256,1024],block_number=3)
    X = identity_block(X,Filters=[256,256,1024],block_number=3)
    X = identity_block(X,Filters=[256,256,1024],block_number=3)
    X = identity_block(X,Filters=[256,256,1024],block_number=3)
    X = identity_block(X,Filters=[256,256,1024],block_number=3)
    X = identity_block(X,Filters=[256,256,1024],block_number=3)
    
    #Conv5_x
    X = convolutional_block(X,Filters=[512,512,2048],block_number=4)
    X = identity_block(X,Filters=[512,512,2048],block_number=4)
    X = identity_block(X,Filters=[512,512,2048],block_number=4)
    
    # Avg Pool
    X = AveragePooling2D((2,2))(X)
    
    # OutPut Layer
    X = Flatten()(X)
    X = Dense(34,activation='softmax')(X)
    model = Model(inputs=X_input,outputs=X)
    return model

In [11]:
input_size=[224,224,3]
model = resnet152(input_size)
model.summary()

Shape of X_input (None, 224, 224, 3)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 55, 55, 64)   0           Conv1[0][0]                      
_______________________________________________________

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(training_data, epochs = 10,batch_size=32)

Epoch 1/10
155/155 [==============================] - 1162s 7s/step - loss: 5.3565 - accuracy: 0.0614
Epoch 2/10
155/155 [==============================] - 1109s 7s/step - loss: 2.8794 - accuracy: 0.2012
Epoch 3/10
155/155 [==============================] - 1079s 7s/step - loss: 2.3167 - accuracy: 0.3481
Epoch 4/10
155/155 [==============================] - 1204s 8s/step - loss: 1.8430 - accuracy: 0.4822
Epoch 5/10
155/155 [==============================] - 1181s 8s/step - loss: 1.4877 - accuracy: 0.5739
Epoch 6/10
155/155 [==============================] - 1122s 7s/step - loss: 1.2484 - accuracy: 0.6428
Epoch 7/10
155/155 [==============================] - 1085s 7s/step - loss: 0.9799 - accuracy: 0.7133
Epoch 8/10
155/155 [==============================] - 1017s 7s/step - loss: 0.7773 - accuracy: 0.7609
Epoch 9/10
155/155 [==============================] - 1036s 7s/step - loss: 0.6026 - accuracy: 0.8154
Epoch 10/10
155/155 [==============================] - 1027s 7s/step - loss: 0.505

In [13]:
model.save("Resnet-50_32Batches_10epochs.h5")

/home/rohit/anaconda3/envs/resnet/lib/python3.7/site-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
